In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

Using tensorboardX


In [2]:
#Dataset = get_dataset('neu_det', 'ctdet')

## 对比两个数据集的不同，看看错在哪里

In [3]:
TEST = 'neu'

In [4]:
if TEST == 'pascal':
    args=['ctdet', '--exp_id', 'pascal_resdcn18_384', '--arch', 'resdcn_18', '--dataset', 'pascal', '--load_model', '../models/ctdet_coco_resdcn18.pth', '--batch_size', '1', '--num_epochs', '70', '--gpus', '0', '--lr_step', '45,60']
else:# TEST == 'neu':
    args=['ctdet', '--exp_id', 'neu_resdcn_18', '--arch', 'resdcn_18', '--batch_size', '1', '--dataset', 'neu_det', '--load_model', '../models/ctdet_coco_resdcn18.pth', '--gpus', '0']

opt = opts().parse(args)
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test

Fix size testing.
training chunk_sizes: [1]
The output will be saved to  /home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../exp/ctdet/neu_resdcn_18
heads {'hm': 6, 'wh': 2, 'reg': 2}


In [5]:
print(opt)



Namespace(K=100, aggr_weight=0.0, agnostic_ex=False, arch='resdcn_18', aug_ddd=0.5, aug_rot=0, batch_size=1, cat_spec_wh=False, center_thresh=0.1, chunk_sizes=[1], data_dir='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data', dataset='neu_det', debug=0, debug_dir='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../exp/ctdet/neu_resdcn_18/debug', debugger_theme='white', demo='', dense_hp=False, dense_wh=False, dep_weight=1, dim_weight=1, down_ratio=4, eval_oracle_dep=False, eval_oracle_hm=False, eval_oracle_hmhp=False, eval_oracle_hp_offset=False, eval_oracle_kps=False, eval_oracle_offset=False, eval_oracle_wh=False, exp_dir='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../exp/ctdet', exp_id='neu_resdcn_18', fix_res=True, flip=0.5, flip_test=False, gpus=[0], gpus_str='0', head_conv=64, heads={'hm': 6, 'wh': 2, 'reg': 2}, hide_data_time=False, hm_hp=True, hm_hp_weight=1, hm_weight=1, hp_weight=1, input_h=200, input_res=200, inpu

In [6]:
logger = Logger(opt)
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

In [7]:
print('Creating model...')
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(
        model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

Creating model...
=> loading pretrained model https://download.pytorch.org/models/resnet18-5c106cde.pth
=> init deconv weights from normal distribution
loaded ../models/ctdet_coco_resdcn18.pth, epoch 140
Skip loading parameter hm.2.weight, required shapetorch.Size([6, 64, 1, 1]), loaded shapetorch.Size([80, 64, 1, 1]).
Skip loading parameter hm.2.bias, required shapetorch.Size([6]), loaded shapetorch.Size([80]).


In [8]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [9]:
print('Setting up data...')
val_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'val'), 
    batch_size=1, 
    shuffle=False,
    num_workers=1,
    pin_memory=True
)
if opt.test:
    _, preds = trainer.val(0, val_loader)
    val_loader.dataset.run_eval(preds, opt.save_dir)

train_loader = torch.utils.data.DataLoader(
    Dataset(opt, 'train'), 
    batch_size=opt.batch_size, 
    shuffle=True,
    num_workers=opt.num_workers,
    pin_memory=True,
    drop_last=True
)

Setting up data...
==> initializing pascal val data.
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Loaded val 180 samples
==> initializing pascal train data.
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loaded train 1620 samples


In [10]:
print('Starting training...')
best = 1e10
for epoch in range(start_epoch + 1, opt.num_epochs + 1):
  mark = epoch if opt.save_all else 'last'
  log_dict_train, _ = trainer.train(epoch, train_loader)
  logger.write('epoch: {} |'.format(epoch))
  for k, v in log_dict_train.items():
    logger.scalar_summary('train_{}'.format(k), v, epoch)
    logger.write('{} {:8f} | '.format(k, v))
  if opt.val_intervals > 0 and epoch % opt.val_intervals == 0:
    save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(mark)), 
                epoch, model, optimizer)
    with torch.no_grad():
      log_dict_val, preds = trainer.val(epoch, val_loader)
    for k, v in log_dict_val.items():
      logger.scalar_summary('val_{}'.format(k), v, epoch)
      logger.write('{} {:8f} | '.format(k, v))
    if log_dict_val[opt.metric] < best:
      best = log_dict_val[opt.metric]
      save_model(os.path.join(opt.save_dir, 'model_best.pth'), 
                  epoch, model)
  else:
    save_model(os.path.join(opt.save_dir, 'model_last.pth'), 
                epoch, model, optimizer)
  logger.write('\n')
  if epoch in opt.lr_step:
    save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(epoch)), 
                epoch, model, optimizer)
    lr = opt.lr * (0.1 ** (opt.lr_step.index(epoch) + 1))
    print('Drop LR to', lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
logger.close()

Starting training...
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/crazing_225.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/patches_8.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/inclusion_173.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/crazing_299.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/patches_234.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/inclusion_102.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterNet/src/lib/../../data/NEU-DET/IMAGES/pitted_surface_283.jpg'
!##DEBUG##!: img_path='/home/xu/NNdev/Deep_learning_Zoo/xingyizhou_CenterN

RuntimeError: The size of tensor a (56) must match the size of tensor b (50) at non-singleton dimension 3